In [1]:
import torch


ModuleNotFoundError: No module named 'torch'

In [ ]:
class FITExplainer:
    def __init__(self, model, generator=None,activation=torch.nn.Softmax(-1),n_classes=2):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.generator = generator
        self.base_model = model.to(self.device)
        self.activation = activation
        self.n_classes=n_classes

    def fit_generator(self, generator_model, train_loader, test_loader, n_epochs=300,cv=0):
        train_joint_feature_generator(generator_model, train_loader, test_loader, generator_type='joint_generator',
                                      n_epochs=300, lr=0.001, weight_decay=0,cv=cv)
        self.generator = generator_model.to(self.device)

    def attribute(self, x, y, n_samples=10, retrospective=False, distance_metric='kl',subsets=None):
        """
        Compute importance score for a sample x, over time and features
        :param x: Sample instance to evaluate score for. Shape:[batch, features, time]
        :param n_samples: number of Monte-Carlo samples
        :return: Importance score matrix of shape:[batch, features, time]
        """
        self.generator.eval()
        self.generator.to(self.device)
        x = x.to(self.device)
        _, n_features, t_len = x.shape
        score = np.zeros(list(x.shape))
        if retrospective:
            p_y_t = self.activation(self.base_model(x))

        for t in range(1, t_len):
            if not retrospective:
                p_y_t = self.activation(self.base_model(x[:, :, :t+1]))
                p_tm1 = self.activation(self.base_model(x[:,:,0:t]))

            for i in range(n_features):
                x_hat = x[:,:,0:t+1].clone()
                div_all=[]
                t1_all=[]
                t2_all=[]
                for _ in range(n_samples):
                    x_hat_t, _ = self.generator.forward_conditional(x[:, :, :t], x[:, :, t], [i])
                    x_hat[:, :, t] = x_hat_t
                    y_hat_t = self.activation(self.base_model(x_hat))
                    if distance_metric == 'kl':
                        if type(self.activation).__name__==type(torch.nn.Softmax(-1)).__name__:
                            div = torch.sum(torch.nn.KLDivLoss(reduction='none')(torch.log(p_tm1), p_y_t), -1) - \
                             torch.sum(torch.nn.KLDivLoss(reduction='none')(torch.log(y_hat_t), p_y_t), -1)
                            lhs = torch.sum(torch.nn.KLDivLoss(reduction='none')(torch.log(p_tm1), p_y_t), -1)
                            rhs = torch.sum(torch.nn.KLDivLoss(reduction='none')(torch.log(y_hat_t), p_y_t), -1)
                            # div = torch.where(rhs>lhs, torch.zeros(rhs.shape), rhs/lhs)
                        else:
                            t1 = kl_multilabel(p_y_t, p_tm1)
                            t2 = kl_multilabel(p_y_t, y_hat_t)
                            div,_ = torch.max(t1 - t2,dim=1)
                            #div = div[:,0] #flatten
                        div_all.append(div.cpu().detach().numpy())
                    elif distance_metric == 'mean_divergence':
                        div = torch.abs(y_hat_t - p_y_t)
                        div_all.append(np.mean(div.detach().cpu().numpy(), -1))
                    elif distance_metric=='LHS':
                        div = torch.sum(torch.nn.KLDivLoss(reduction='none')(torch.log(p_tm1), p_y_t), -1)
                        div_all.append(div.cpu().detach().numpy())
                    elif distance_metric=='RHS':
                        div = torch.sum(torch.nn.KLDivLoss(reduction='none')(torch.log(y_hat_t), p_y_t), -1)
                        div_all.append(div.cpu().detach().numpy())
                E_div = np.mean(np.array(div_all),axis=0)
                if distance_metric =='kl':
                    # score[:, i, t] = E_div
                    score[:, i, t] = 2./(1+np.exp(-5*E_div)) - 1
                elif distance_metric=='mean_divergence':
                    score[:, i, t] = 1-E_div
                else:
                    score[:, i, t] = E_div
        return score